# Cybershuttle Molecular Dynamics SDK

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease. In this notebook, we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution. The experiment object `exp` is an instance of `cybershuttle.base.Experiment`.

In [1]:
import cybershuttle as cs
from cybershuttle import md

## Authenticate for Remote Execution

To authenticate for remote execution, use the `cs.auth.login()` method. This method will prompt you to enter your credentials and authenticate your session.

In [2]:
cs.auth.login()
token = cs.auth.context.access_token

User code: FHBR-GIYT
Please authenticate by visiting: https://auth.cybershuttle.org/realms/10000000/device?user_code=FHBR-GIYT
Waiting for authorization...
Authorization successful!


## Define a NAMD Experiment

To define a NAMD experiment, we initialize an instance of the `cybershuttle.base.Experiment` class with the required parameters such as the name of the experiment, and the input PDB and PSF files. The experiment object `exp` is created as shown below:

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36m_prot.prm"
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ]
)

## Add Replicas for NAMD Experiment

To add replicas to the NAMD experiment, we use the `add_replica()` method of the `exp` object. This method adds a replica to the experiment. In this example, we add 4 replicas as shown below:

In [ ]:
runtimes = cs.runtime.query()

# always 0th index is local runtime inside
# 0th runtime requires docker to be installed

display(runtimes)

In [ ]:
# choose between any replicas user has access to
# exp.add_replica(*runtimes)

# OR choose any replica that is not local
exp.add_replica(*runtimes[1:])
exp.add_replica(*runtimes[1:])

# OR use only a specific replica
# exp.add_replica(runtimes[1])

## Create Execution Plan

To create an execution plan for the experiment, use the `plan()` method of the `exp` object. This method will generate a plan for the experiment, which can then be described using the `describe()` method. The code to create and describe the plan is shown below:

In [ ]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan
plan.save_json("plan.json")  # save the plan state

## Execute the Plan

In [ ]:
plan.launch()
plan.save_json("plan.json")  # save the plan state

## Option A - Wait for Completion

In [3]:
plan = cs.plan.Plan.load_json("plan.json")
plan.describe()

Task(name=yasith_namd_experiment_9801, app_id=NAMD, inputs={'MD-Instructions-Input': 'data/pull.conf', 'Coordinates-PDB-File': 'data/structure.pdb', 'Protein-Structure-File_PSF': 'data/structure.psf', 'FF-Parameter-Files': ['data/par_all36_water.prm', 'data/par_all36m_prot.prm'], 'Execution_Type': 'CPU', 'Optional_Inputs': ['data/b4pull.pdb', 'data/b4pull.restart.coor', 'data/b4pull.restart.vel', 'data/b4pull.restart.xsc'], 'Number of Replicas': 1}, runtime=Remote(args={'cluster': 'login.expanse.sdsc.edu'}))
Task(name=yasith_namd_experiment_58A2, app_id=NAMD, inputs={'MD-Instructions-Input': 'data/pull.conf', 'Coordinates-PDB-File': 'data/structure.pdb', 'Protein-Structure-File_PSF': 'data/structure.psf', 'FF-Parameter-Files': ['data/par_all36_water.prm', 'data/par_all36m_prot.prm'], 'Execution_Type': 'CPU', 'Optional_Inputs': ['data/b4pull.pdb', 'data/b4pull.restart.coor', 'data/b4pull.restart.vel', 'data/b4pull.restart.xsc'], 'Number of Replicas': 1}, runtime=Remote(args={'cluster': 

In [6]:
plan.launch_agents()

Launching agents...
[Task] Executing yasith_namd_experiment_9801_agent on Remote(args={'cluster': 'login.expanse.sdsc.edu'})
[Remote] Experiment Created: name=yasith_namd_experiment_9801_agent
[AV] Preprocessing args...
[AV] Validating args...


AssertionError: 

## Option B - Terminate Execution

In [ ]:
plan.stop()

## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [ ]:
for task in plan.tasks:
    status = task.status()
    files = task.files()
    print(status, files)

Displaying the intermediate results generated by each replica (task)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()

In [ ]:
from cybershuttle.airavata import AiravataOperator

assert token is not None

av = AiravataOperator(token)

# Example: get app interface id and storage resource id
app_interface_id = av.get_app_interface_id("NAMD")
storage = av.get_storage()
storage_resource_id = storage.storageResourceId
display(f"application_interface_id={app_interface_id}")
display(f"storage_resource_id={storage_resource_id}")

# Example: upload files
experiment_name = "NAMD_MD_SDK"
app_name = "NAMD"
hpc_name = "login.expanse.sdsc.edu"

experiment = av.launch_experiment(
    experiment_name=experiment_name,
    app_name=app_name,
    computation_resource_name=hpc_name,
    inputs={
        "MD-Instructions-Input": "./data/pull.conf",
        "Constraints-PDB-File": "./data/structure.pdb",
        "Protein-Structure-File_PSF": "./data/structure.psf",
        "FF-Parameter-Files": [
            "./data/par_all36_water.prm",
            "./data/par_all36m_prot.prm"
        ],
        "Execution_Type": "CPU",
        "Optional_Inputs": [
            "./data/b4pull.restart.xsc",
            "./data/b4pull.restart.vel",
            "./data/b4pull.pdb",
            "./data/b4pull.restart.coor",
        ],
    }
)